## Библиотеки и настройки

In [1]:
# импорт библиотек

import pandas as pd

# регулярки
import re

from datetime import datetime

# прогресс-бар
from tqdm.notebook import trange, tqdm


import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from pymystem3 import Mystem
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# путь к файлам

file_path = 'C:/00_Projects/RIA/2022/02_elk_accidents/00_data/'

### Выгружаю датасет

In [6]:
# выгружаю файл в датафрейм

work = pd.read_excel(file_path + 'animal_accidents.xlsx', sheet_name='regions')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/00_Projects/RIA/2022/02_elk_accidents/00_data/animal_accidents.xlsx'

In [ ]:
# смотрю случайные строки

work.sample(3)

In [ ]:
# количество строк и столбцов

work.shape

In [ ]:
# типы данных

work.dtypes

In [ ]:
# проверка на пустые значения

work.isnull().sum()

### Разделяю столбцы

In [ ]:
# дата решения

work['act_date'] = work['act_name'].str.replace(r'.+ от ', ' ', regex=True)
work['act_date'] = work['act_date'].str.replace(r' г\. .+', ' ', regex=True)
work['act_date'] = work['act_date'].str.replace(r' г\.', ' ', regex=True)

# меняю месяц на число
work['act_date'] = work['act_date'].str.replace('января', '-01-', regex=True)
work['act_date'] = work['act_date'].str.replace('февраля', '-02-', regex=True)
work['act_date'] = work['act_date'].str.replace('марта', '-03-', regex=True)
work['act_date'] = work['act_date'].str.replace('апреля', '-04-', regex=True)
work['act_date'] = work['act_date'].str.replace('мая', '-05-', regex=True)
work['act_date'] = work['act_date'].str.replace('июня', '-06-', regex=True)
work['act_date'] = work['act_date'].str.replace('июля', '-07-', regex=True)
work['act_date'] = work['act_date'].str.replace('августа', '-08-', regex=True)
work['act_date'] = work['act_date'].str.replace('сентября', '-09-', regex=True)
work['act_date'] = work['act_date'].str.replace('октября', '-10-', regex=True)
work['act_date'] = work['act_date'].str.replace('ноября', '-11-', regex=True)
work['act_date'] = work['act_date'].str.replace('декабря', '-12-', regex=True)
work['act_date'] = work['act_date'].str.replace(' ', '', regex=True)

In [ ]:
%%time

# вытаскиваю номер дела

def find_act_id(df):
    try:
        act_id = re.findall(r'(?<=дел[о|у] №)\S+', work['act_text'].str.lower())[0]
        
    except:
        act_id = 'none'
        
    return act_id

#work['act_id'] = work.apply(find_act_id, axis=1)

In [ ]:
# считаю длину текста судебных решений

#work['text_len'] = work['act_text'].apply(lambda text: len(text))
#work = work.astype({'text_len': 'int32'})

In [ ]:
# проверяю гипотезу о равенстве названия текста и заголовка в самом тексте

#def split_title_from_act(df):
    
#    return df['act_text'][:len(df['act_name'])]

#work['new_title'] = work.apply(split_title_from_act, axis=1)

#work['test'] = work.act_name == work.new_title

#work.test.value_counts()

#work.drop(['new_title', 'test'], axis=1, inplace=True)

In [ ]:
# удаляю заголовок из текста дела

def delete_title_from_act(df):
    
    return df['act_text'][len(df['act_name']):]

work['act_text'] = work.apply(delete_title_from_act, axis=1)

work['act_text'] = work['act_text'].str.strip()

In [ ]:
# проверяю гипотезу о равенстве названия текста и заголовка в самом тексте

#def split_court_info_from_act(df):
    
#    return df['act_text'][:len(df['court_info'])]

#work['new_court'] = work.apply(split_court_info_from_act, axis=1)

#work['test'] = work.court_info == work.new_court

#work.test.value_counts()

#work.drop(['new_court', 'test'], axis=1, inplace=True)

In [ ]:
# удаляю court_info из текста

def delete_court_info_from_act(df):
    
    return df['act_text'][len(df['court_info']):]

work['act_text'] = work.apply(delete_court_info_from_act, axis=1)

work['act_text'] = work['act_text'].str.strip()

In [ ]:
work.sample(3)

In [ ]:
# делю court_info на два столбца

temp_work = work['court_info'].str.split(' - ', expand=True, n=1)
temp_work.columns = ['court', 'act_info']
work = pd.concat([work, temp_work], axis=1)

In [ ]:
# отщипываю act_text судебную практику (статьи)

temp_work = work['act_text'].str.split('Судебная практика по:', expand=True, n=1)
temp_work.columns = ['act_text', 'rest']
temp_work['rest'] = temp_work['rest'].str.strip()
temp_work['rest'] = temp_work['rest'].str.replace(r' Судебная практика.+ нормы? ', ' ', regex=True) # убираю артефакты
temp_work['rest'] = temp_work['rest'].str.replace('РФ', 'РФstop_word') # делю по статьям
temp_work['rest'] = temp_work['rest'].str.split('stop_word')

work.drop(columns=['act_text'], inplace=True)
work = pd.concat([work, temp_work], axis=1)
work.rename(columns={'rest': 'articles'}, inplace=True)

In [ ]:
# отщипываю act_text другие документы по делу

temp_work = work['act_text'].str.split('Последние документы по делу:', expand=True, n=1)
temp_work.columns = ['act_text', 'rest']
temp_work['rest'] = temp_work['rest'].str.strip()

work.drop(columns=['act_text'], inplace=True)
work = pd.concat([work, temp_work], axis=1)
work.rename(columns={'rest': 'associated_acts'}, inplace=True)

In [ ]:
# список сторон дела

parties_list = ['Суд:', 'Истцы:', 'Подсудимые:', 'Ответчики:', 'Иные лица:', 'Судьи дела:']

In [ ]:
# отщипываю блок с участниками дела

def split_act_parties(df):
    for item in parties_list:
        act_parties = re.findall(f'{item}\s.+', df['act_text'])
        if act_parties != []:
            break
    
    temp_dict = {}
    temp_dict['block_len'] = len(act_parties[0])
    
    act_parties = act_parties[0].strip()
    
    for item in parties_list:
        temp_dict[item] = (re.findall(f'(?<={item} ).*?(?= \(подробнее)', act_parties))
        
            
    return temp_dict

work['act_parties'] = work.apply(split_act_parties, axis=1)

In [ ]:
# удаляю блок из act_text

def split_parties_block_from_act(df):
    
    return df['act_text'][:-(df['act_parties']['block_len'])]

work['act_text'] = work.apply(split_parties_block_from_act, axis=1)

In [ ]:
work.sample(3)

In [ ]:
work.drop(columns=['act_text'], inplace=True)
work = pd.concat([work, temp_work], axis=1)
work.rename(columns={'rest': 'associated_acts'}, inplace=True)

In [ ]:
temp_work.head()

In [ ]:
work.head()

In [ ]:
test = re.findall(r'Суд: .+\(подробнее\)', work.act_text[1456])

In [ ]:
# считаю длину текста судебных решений

temp_work['text_len'] = temp_work['act_text'].apply(lambda text: len(text))
temp_work = temp_work.astype({'text_len': 'int32'})

In [ ]:
temp_work['rest'] = temp_work['rest'].str.strip()

In [ ]:
work = pd.concat([work, temp_work], axis=1)

In [ ]:
re.findall('Последние документы по делу:',)

work.act_text[1456]

In [ ]:
work = work[['act_name', 'act_date', 'court', 'act_info', 'act_text', 'text_len', 'act_url', 'instance', 'queries']]

In [ ]:
work.sample(3)

In [ ]:
re.search(r'Истцы:.+(подробнее)?', work.act_text[6])

In [ ]:
work.act_text[1456]

In [ ]:
test = re.findall(r'Суд: .+\(подробнее\)', work.act_text[1456])

In [ ]:
test[0].split(' (подробнее) ')

In [ ]:
work.act_text[344]

In [ ]:
# создаю датафрейм только с лосями

work_elks = work[work.queries.str.contains('лос', case=False)]

In [ ]:
work_elks.shape

In [ ]:
work.head(1)

In [ ]:
work.

In [ ]:
temp = work.act_url.value_counts().reset_index()

In [ ]:
temp = temp[temp.act_url > 1]['index'].to_list()

In [ ]:
temp_work = pd.DataFrame()

for i in temp:
    temp_work = temp_work.append(work[work.act_url == i])
    

In [ ]:
list(work.act_date.unique())

In [ ]:
work_temp = work[~(
    
    (work.act_text.str.contains(' лося', case=False)) |
    (work.act_text.str.contains(' лосем', case=False)) |
    (work.act_text.str.contains(' лосенка', case=False)) |
    (work.act_text.str.contains(' лосих', case=False)) |
    (work.act_text.str.contains(' лось', case=False)) |
    (work.act_text.str.contains(' медвед', case=False))
    
    
)]

In [ ]:
work_temp

In [ ]:
work.sample(3)

In [ ]:
work.to_excel(file_path + 'temp_666.xlsx')